In [2]:
import os
import requests
import pandas as pd
import numpy as np
from io import BytesIO
from PIL import Image
from tqdm import tqdm

In [2]:
annot_df = pd.read_csv('/home/ariya/workspace/datasets/animals10-dvc/annotation_df.csv')

In [3]:
test_df = annot_df[annot_df['split']=='test']
len(test_df)

2618

In [4]:
n_sample = 500
sample_df = test_df.sample(n=n_sample, random_state=42)
img_paths = list(sample_df['abs_path'])
img_paths[:5]

['/home/ariya/workspace/datasets/animals10-dvc/images/test/chicken/OIP-Gfaxns6HA4SxAJGyDKMyfAHaFi.jpeg',
 '/home/ariya/workspace/datasets/animals10-dvc/images/test/spider/OIP-y8GPU8H1a3IvDSJhat_KNQHaFj.jpeg',
 '/home/ariya/workspace/datasets/animals10-dvc/images/test/cow/OIP-YYexVfC98d1zMJGxK1QTJAHaEo.jpeg',
 '/home/ariya/workspace/datasets/animals10-dvc/images/test/spider/eb33b7092af2003ed1584d05fb1d4e9fe777ead218ac104497f5c97ca5ecb3b9_640.jpeg',
 '/home/ariya/workspace/datasets/animals10-dvc/images/test/dog/OIP-DU8oP5HWLQpETU4hP_VmYQHaEK.jpeg']

In [4]:
service_host = 'nginx'
service_port = '80'

## post with sample from test set

In [6]:
for img_path in tqdm(img_paths):
    res = requests.post(f'http://{service_host}:{service_port}/predict/',
                    files={'file':open(img_path, 'rb')}
                    )
    if res.status_code != 200:
        print(f'Something went wrong with {img_path}')

100%|██████████| 500/500 [05:54<00:00,  1.41it/s]


## post with noise images

In [5]:
for img_path in tqdm(range(500)):
    noise = np.random.randint(0, 255, size=(200,200,3), dtype=np.uint8)
    pil_img = Image.fromarray(noise)
    byte_io = BytesIO()
    pil_img.save(byte_io, format="JPEG")
    byte_io.seek(0)
    
    res = requests.post(f'http://{service_host}:{service_port}/predict/',
                    files={'file':byte_io}
                    )
    if res.status_code != 200:
        print(f'Something went wrong with {img_path}')

100%|██████████| 500/500 [05:22<00:00,  1.55it/s]
